In [1]:
import polars as pl
import os
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display_html
InteractiveShell.ast_node_interactivity = "all"

auhouseprices_dir = '../data/raw/auhouseprices/'

In [2]:
# Read each .csv file in the directory into a single polars DataFrame
df = pl.DataFrame({})
for file in os.listdir(auhouseprices_dir):
    if file.endswith('.csv'):
        df = df.vstack(pl.read_csv(auhouseprices_dir + file))


In [10]:
# Count number of null or missing values in each column
df.null_count()